#**스마트폰 센서 데이터 기반 모션 분류**
# 단계3 : 단계별 모델링


## 0.미션

단계별로 나눠서 모델링을 수행하고자 합니다.  

* 단계1 : 정적(0), 동적(1) 행동 분류 모델 생성
* 단계2 : 세부 동작에 대한 분류모델 생성
    * 단계1 모델에서 0으로 예측 -> 정적 행동 3가지 분류 모델링
    * 단계1 모델에서 1으로 예측 -> 동적 행동 3가지 분류 모델링 
* 모델 통합
    * 두 단계 모델을 통합하고, 새로운 데이터에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
* 성능 비교
    * 기본 모델링의 성능과 비교
    * 모든 모델링은 [다양한 알고리즘 + 성능 튜닝]을 수행해야 합니다.


## 1.환경설정

### (1) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# 필요하다고 판단되는 라이브러리를 추가하세요.
from sklearn.model_selection import train_test_split

### (2) 데이터 불러오기

* 주어진 데이터셋
    * data01_train.csv : 학습 및 검증용

 <br/>  

* 세부 요구사항
    - data01_train.csv 를 불러와 'data' 이름으로 저장합니다.
        - data에서 변수 subject는 삭제합니다.
    - data01_test.csv 를 불러와 'new_data' 이름으로 저장합니다.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/딥러닝/5차 미프/data01_train.csv")
data.drop("subject", axis = 1, inplace = True)

## 2.데이터 전처리

* 세부 요구사항
    - Label 추가 : data 에 Activity_dynamic 를 추가합니다. Activity_dynamic은 과제1에서 is_dynamic과 동일한 값입니다.
    - x와 y1, y2로 분할하시오.
        * y1 : Activity
        * y2 : Activity_dynamic
    - train : val = 8 : 2 혹은 7 : 3
    - random_state 옵션을 사용하여 다른 모델과 비교를 위해 성능이 재현되도록 합니다.

In [ ]:
data['is_dynamic'] = data['Activity'].apply(lambda x: 1 if x in ['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS'] else 0)

x = data.drop(['Activity', 'is_dynamic'], axis = 1)
y1 = data['Activity']
y2 = data['is_dynamic']

x_train, x_val, y1_train, y1_val, y2_train, y2_val = train_test_split(x, y1, y2, test_size = 0.2, random_state = 42)

## **3.단계별 모델링**

![](https://github.com/DA4BAM/image/blob/main/step%20by%20step.png?raw=true)

### (1) 단계1 : 정적/동적 행동 분류 모델

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)과 동적 행동(동적 : Walking, Walking-Up, Walking-Down)을 구분하는 모델 생성.
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

#### 1) 알고리즘1 : RandomForest

In [ ]:
x = data.drop(['Activity', 'is_dynamic'], axis = 1)
y2 = data['is_dynamic']
# y1 = data['Activity']
# x_train, x_val, y1_train, y1_val, y2_train, y2_val = train_test_split(x, y1, y2, test_size = 0.2, random_state = 42)

In [ ]:
x_train.shape, y2_train.shape

((4704, 561), (4704,))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

model = RandomForestClassifier()

model.fit(x_train, y2_train)

y2_pred = model.predict(x_val)

In [ ]:
print(f"정확도 : {accuracy_score(y2_pred, y2_val)}")
print(classification_report(y2_pred, y2_val))

정확도 : 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       657
           1       1.00      1.00      1.00       520

    accuracy                           1.00      1177
   macro avg       1.00      1.00      1.00      1177
weighted avg       1.00      1.00      1.00      1177



#### 2) 알고리즘2 : XGBoost

In [ ]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier()

model_xgb.fit(x_train, y2_train)

y2_pred = model.predict(x_val)

print(f"정확도 : {accuracy_score(y2_pred, y2_val)}")
print(classification_report(y2_pred, y2_val))

정확도 : 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       657
           1       1.00      1.00      1.00       520

    accuracy                           1.00      1177
   macro avg       1.00      1.00      1.00      1177
weighted avg       1.00      1.00      1.00      1177



### (2) 단계2-1 : 정적 동작 세부 분류

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)인 데이터 추출
    * Laying, Sitting, Standing 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

![](https://github.com/DA4BAM/image/blob/main/step%20by%20step.png?raw=true)

In [ ]:
static_data = data.loc[data["Activity"].isin(["LAYING", "SITTING", "STANDING"])]

In [ ]:
# static_data.drop("is_dynamic", axis = 1, inplace = True)

<ipython-input-59-6f80e828d9d6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_data.drop("is_dynamic", axis = 1, inplace = True)


In [ ]:
static_x = static_data.drop("Activity", axis = 1)
static_y = static_data.loc[:, "Activity"]

In [ ]:
x_train_sta, x_test_sta, y_train_sta, y_test_sta = train_test_split(static_x, static_y, test_size = 0.2, random_state = 42)

In [ ]:
model_static = RandomForestClassifier()

model_static.fit(x_train_sta, y_train_sta)

y_sta_pred = model_static.predict(x_test_sta)

print(f"정확도 : {accuracy_score(y_sta_pred, y_test_sta)}")
print(classification_report(y_sta_pred, y_test_sta))

정확도 : 0.9768160741885626
              precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00       221
     SITTING       0.97      0.96      0.96       208
    STANDING       0.96      0.97      0.97       218

    accuracy                           0.98       647
   macro avg       0.98      0.98      0.98       647
weighted avg       0.98      0.98      0.98       647



### (3) 단계2-2 : 동적 동작 세부 분류

* 세부 요구사항
    * 동동적 행동(Walking, Walking Upstairs, Walking Downstairs)인 데이터 추출
    * Walking, Walking Upstairs, Walking Downstairs 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [ ]:
data["Activity"].value_counts()

LAYING                1115
STANDING              1087
SITTING               1032
WALKING                998
WALKING_UPSTAIRS       858
WALKING_DOWNSTAIRS     791
Name: Activity, dtype: int64

In [ ]:
dynamic_data = data.loc[data["Activity"].isin(["WALKING", "WALKING_UPSTAIRS", "WALKING_DOWNSTAIRS"])]

In [ ]:
dynamic_data["Activity"].value_counts()

WALKING               998
WALKING_UPSTAIRS      858
WALKING_DOWNSTAIRS    791
Name: Activity, dtype: int64

In [ ]:
dynamic_x = dynamic_data.drop("Activity", axis = 1)
dynamic_y = dynamic_data.loc[:, "Activity"]
x_train_dyn, x_test_dyn, y_train_dyn, y_test_dyn = train_test_split(dynamic_x, dynamic_y, test_size = 0.2, random_state = 42)

In [ ]:
model_dynamic = RandomForestClassifier()

model_dynamic.fit(x_train_dyn, y_train_dyn)

y_dyn_pred = model_dynamic.predict(x_test_dyn)

print(f"정확도 : {accuracy_score(y_dyn_pred, y_test_dyn)}")
print(classification_report(y_dyn_pred, y_test_dyn))

정확도 : 0.9811320754716981
                    precision    recall  f1-score   support

           WALKING       0.98      0.98      0.98       196
WALKING_DOWNSTAIRS       0.98      0.98      0.98       163
  WALKING_UPSTAIRS       0.98      0.98      0.98       171

          accuracy                           0.98       530
         macro avg       0.98      0.98      0.98       530
      weighted avg       0.98      0.98      0.98       530



### (4) 분류 모델 합치기


* 세부 요구사항
    * 두 단계 모델을 통합하고, 새로운 데이터(test)에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
    * 데이터 파이프라인 구축 : test데이터가 로딩되어 전처리 과정을 거치고, 예측 및 성능 평가 수행

![](https://github.com/DA4BAM/image/blob/main/pipeline%20function.png?raw=true)

#### 1) 함수 만들기